In [1]:
import pickle
from collections import namedtuple

import os
from urllib.parse import parse_qs

import ipywidgets as widgets
import ipyvuetify as v
import traitlets
import ipyvuedraggable as vd

import pandas as pd

Peiling = namedtuple('Peiling', ['verwacht', 'laag', 'hoog'])

In [2]:
with open('peilingen.pkl', 'rb') as fh:
    numbers = pickle.load(fh)
with open('table.pkl', 'rb') as fh:
    table = pickle.load(fh)
with open('candidates.pkl', 'rb') as fh:
    candidates = pickle.load(fh)
with open('candidate_plaatsen.pkl', 'rb') as fh:
    places = pickle.load(fh)

parties = list(numbers.keys())

In [7]:
buttons = {party: v.Btn(text=True, style_="text-transform: none", children=[party]) for party in parties}
toggle_multi = v.BtnToggle(v_model=url_picked_party_indices, multiple=True, children=list(buttons.values()))

In [9]:
candidates_df = pd.DataFrame(candidates)

In [10]:
candidate_list_widgets = {}
for party in parties:
    party_candidates = candidates_df[candidates_df.partij == party]
    candidate_list_widgets[party] = v.List(children=[
        v.Subheader(children=[party]),
        v.ListItemGroup(v_model="selectedItem", children=[
            v.ListItem(children=[
                v.ListItemContent(children=[
                    v.ListItemTitle(children=[naam])
                ])
            ])
            for naam in party_candidates.naam
        ])
    ])

In [11]:
lists_layout = v.Layout()

In [12]:
def on_toggle(btn_toggle, event, data):
    selected = tuple(parties[ix] for ix in sorted(data))
    peiling = table[selected]

    seats.low = peiling.laag
    seats.exp = peiling.verwacht
    seats.high = peiling.hoog
    
    lists_layout.children = [candidate_list_widgets[party] for party in selected]
    candidate_list_widgets['CU']
    
    url = base_url
    if len(selected) > 0:
        url = base_url + "?" + "parties=" + ",".join(selected)

    share_url.value = f'<a href="{url}">{url}</a>'

toggle_multi.on_event('change', on_toggle)
if len(url_picked_party_indices) > 0:
    on_toggle(None, None, url_picked_party_indices)

display(seats)
display(v.Layout(children=[toggle_multi]))
display(lists_layout)

SeatsWidget()

Layout(children=[BtnToggle(children=[Btn(children=['VVD'], style_='text-transform: none', text=True), Btn(chil…

Layout()

In [13]:
ministeries = ["Algemene Zaken (Minister-president)",
               "Binnenlandse Zaken en Koninkrijksrelaties",
               "Buitenlandse Zaken",
               "Defensie",
               "Economische Zaken en Klimaat",
               "Financiën",
               "Infrastructuur en Waterstaat",
               "Landbouw, Natuur en Voedselkwaliteit",
               "Justitie en Veiligheid",
               "Onderwijs, Cultuur en Wetenschap",
               "Sociale Zaken en Werkgelegenheid",
               "Volksgezondheid, Welzijn en Sport"]

In [14]:
koning = v.CardText(children=[
#     v.Html(children=[
#         '<span class="subheading">Koning</span>'
#     ]),
    v.Subheader(children=["Koning"]),
    v.ChipGroup(column=True, children=[
        v.Chip(children=["Willem-Alexander der Nederlanden"])
    ])
])

chip1 = True
chip_groups = {ministerie: v.CardText(children=[
#     v.Html(children=[
#         f'<span class="subheading">{ministerie}</span>'
#     ]),
    v.Subheader(children=[ministerie]),
    v.ChipGroup(column=True, children=[
        v.Chip(outlined=True, 
               if_="chip1",
               close=True,
               children=["Test"])
    ])
]) for ministerie in ministeries}

def on_click_close(*args, **kwargs):
    chip1 = False

chip_groups['Defensie'].children[1].children[0].on_event('click:close', on_click_close)

regering_bar = v.Toolbar(flat=True, dark=True, color="deep-purple accent-4",
                         children=[v.ToolbarTitle(children=["Regering"])])
    
regering = v.Card(_class="mx-auto", max_width="400",
                  children=[regering_bar, koning] + list(chip_groups.values()))

regering


# TIP MAARTEN: maak 1 widget (met template) en denk in termen van data
# TIP MARIO: https://github.com/mariobuikhuizen/ipyvuedraggable

Card(children=[Toolbar(children=[ToolbarTitle(children=['Regering'])], color='deep-purple accent-4', dark=True…

In [16]:
chip = v.Chip(outlined=True, 
               close=True,
               children=["Test"])

def on_clicky_close(widget, event, data):
    widget.layout.display = 'none'

def on_clicky(*args, **kwargs):
    print("clikass")

chip.on_event('click:close', on_clicky_close)
chip.on_event('click', on_clicky)

display(chip)

Chip(children=['Test'], close=True, outlined=True)

clikass
clikass


In [17]:
class ClosableChip(v.VuetifyTemplate):
    chip = traitlets.Bool(True).tag(sync=True)
    template = traitlets.Unicode('''
<template>
    <v-chip
      v-if="chip"
      close
      outlined
      @click:close="chip = false"
    >
</template>
''').tag(sync=True)

chip = ClosableChip()
chip

ClosableChip()

Druk op de knoppen om partijen toe te voegen aan je coalitie. Een meerderheid heeft 76 zetels nodig. De helder rode zetel is het verwachte aantal zetels. In licht rood (zijwaarts gekantelde zetels) zie je onder- en bovengrenzen van het verwachte aantal zetels op basis van verschillende peilingen.

Press the buttons to add parties to your coalition. A majority coalition needs 76 seats. The bright red seat is the expected number of seats. In light red (tipped sideways seats) we show the lower and upper limits of the expected amount of seats based on different polls.

In [18]:
display(share_url)

HTML(value='<a href="http://localhost">http://localhost</a>', description='Share link:')

---

Data: [Peilingwijzer (op basis van peilingen I&O Research, Ipsos en Kantar)](https://peilingwijzer.tomlouwerse.nl/). Code: [see blogpost on Medium](https://blog.esciencecenter.nl/coalition-polls-for-the-people-with-coalitiewijzer-68bca83b95e7).

By [Patrick Bos](https://twitter.com/egpbos/)

[<img src="logo.png" alt="Netherlands eScience Center logo" style="height: 1.5em;" align="left"/>](https://esciencecenter.nl/)

In [4]:
import ipyvuetify as v
import ipyvuedraggable as d
from traitlets import (Any, Unicode, List)

# Output a dummy Draggable instance, so ipyvue gets loaded. This does not 
# happen automatically when only VueTemplate is used
d.Draggable()

Draggable()

In [5]:
def getItems():
    return [
        {
          'id': 1,
          'avatar': "https://s3.amazonaws.com/vuetify-docs/images/lists/1.jpg",
          'title': "Brunch this life?",
          'subtitle': "Subtitle 1"
        },
        {
          'id': 2,
          'avatar': "https://s3.amazonaws.com/vuetify-docs/images/lists/2.jpg",
          'title': "Winter Lunch",
          'subtitle': "Subtitle 2"
        },
        {
          'id': 3,
          'avatar': "https://s3.amazonaws.com/vuetify-docs/images/lists/3.jpg",
          'title': "Oui oui",
          'subtitle': "Subtitle 3"
        }
      ]

def getItems2():
    return [
    {
      'id': 4,
      'avatar': "https://s3.amazonaws.com/vuetify-docs/images/lists/4.jpg",
      'title': "Brunch this weekend?",
      'subtitle': "Subtitle 4"
    },
    {
      'id': 5,
      'avatar': "https://s3.amazonaws.com/vuetify-docs/images/lists/5.jpg",
      'title': 'Summer BBQ',
      'subtitle': "Subtitle 5"
    }
]

In [6]:
class MyDraggable(v.VuetifyTemplate):
    items =  List(getItems()).tag(sync=True)

    items2 = List(getItems2()).tag(sync=True)

    template = Unicode('''
        <v-content>
          <v-container fluid>
            <v-layout align-start justify-center>
              <v-flex xs4 class="elevation-1 pa-3 ma-2">
                <v-list two-line>
                  <v-subheader>
                    FIRST LIST
                  </v-subheader>
                  <draggable v-model="items" :group="{name:'people'}" style="min-height: 10px">
                    <template v-for="item in items">
                    <v-list-item :key="item.id">
                    <v-list-item-avatar>
                      <img :src="item.avatar">
                    </v-list-item-avatar>
                    <v-list-item-content>
                      <v-list-item-title v-html="item.title"></v-list-item-title>
                      <v-list-item-sub-title v-html="item.subtitle"></v-list-item-sub-title>
                    </v-list-item-content>
                  </v-list-item>
                </template>
                  </draggable>
                </v-list>
              </v-flex>
              <v-flex xs4 class="elevation-1 pa-3 ma-2">
                <v-list two-line>
                  <v-subheader>
                    SECOND LIST
                  </v-subheader>
                  <draggable v-model="items2" :group="{name:'people'}" style="min-height: 10px">
                    <template v-for="item in items2">
                  <v-list-item :key="item.id">
                    <v-list-item-avatar>
                      <img :src="item.avatar">
                    </v-list-item-avatar>
                    <v-list-item-content>
                      <v-list-item-title v-html="item.title"></v-list-item-title>
                      <v-list-item-sub-title v-html="item.subtitle"></v-list-item-sub-title>
                    </v-list-item-content>
                  </v-list-item>
                </template>
                  </draggable>
                </v-list>
              </v-flex>
            </v-layout>
          </v-container>
        </v-content>
        ''').tag(sync=True)
    
    
MyDraggable()

MyDraggable(items=[{'id': 1, 'avatar': 'https://s3.amazonaws.com/vuetify-docs/images/lists/1.jpg', 'title': 'B…

In [7]:
def makeListItem(item):
    return v.ListItem(children=[
        v.ListItemAvatar(children=[
            v.Html(tag='img', attributes={'src': item['avatar']})
        ]),
        v.ListItemContent(children=[
            v.ListItemTitle(children=[item['title']]),
            v.ListItemSubtitle(children=[item['subtitle']])
        ])
    ])

dg1 = d.Draggable(
    v_model=getItems(),
    group={'name': 'people'},
    children=[makeListItem(item) for item in getItems()])

def update_dg1(change):
    dg1.children=[makeListItem(item) for item in dg1.v_model]
    
dg1.observe(update_dg1, names=['v_model'])


dg2 = d.Draggable(
    v_model=getItems2(),
    group={'name': 'people'},
    children=[makeListItem(item) for item in getItems2()])

def update_dg2(change):
    dg2.children=[makeListItem(item) for item in dg2.v_model]
    
dg2.observe(update_dg2, names=['v_model'])


v.Content(children=[
    v.Container(fluid=True, children=[
        v.Layout(align_start=True, justify_center=True, children=[
            v.Flex(xs4=True, class_='elevation-1 pa-3 ma-2', children=[
                v.List(two_line=True, children=[
                    v.Subheader(children=['FIRST LIST']),
                    dg1
                ])
            ]),
            v.Flex(xs4=True, class_='elevation-1 pa-3 ma-2', children=[
                v.List(two_line=True, children=[
                    v.Subheader(children=['SECOND LIST']),
                    dg2
                ])
            ])
        ])
    ])
])


Content(children=[Container(children=[Layout(align_start=True, children=[Flex(children=[List(children=[Subhead…

In [ ]:
class MyDraggableArea(v.VuetifyTemplate):
    items =  List(getItems() + getItems2()).tag(sync=True)

    items2 = List().tag(sync=True)

    template = Unicode('''
        <!-- Toplevel draggable is used to cancel drop -->
        <draggable id="catchAll" class="droptarget" :group="{name:'people'}">
        <v-content>
          <v-container fluid>
            <v-layout align-start justify-center>
              <v-flex xs4 class="elevation-1 pa-3 ma-2">
                <v-list two-line>
                  <v-subheader>
                    FIRST LIST
                  </v-subheader>
                  <draggable
                      id="source"
                      :move="checkMove"
                      v-model="items" 
                      :group="{name:'people', pull:'clone'}"
                      style="min-height: 10px">
                    <template v-for="item in items">
                    <v-list-item :key="item.id">
                    <v-list-item-avatar>
                      <img :src="item.avatar">
                    </v-list-item-avatar>
                    <v-list-item-content>
                      <v-list-item-title v-html="item.title"></v-list-item-title>
                      <v-list-item-subtitle v-html="item.subtitle"></v-list-item-sub-title>
                    </v-list-item-content>
                  </v-list-item>
                </template>
                  </draggable>
                </v-list>
              </v-flex>
              <v-flex xs4 class="elevation-1 pa-3 ma-2">
                <v-list two-line>
                  <v-subheader>
                    DROP AREA
                  </v-subheader>
                  <draggable v-model="items2" :group="{name:'people'}" class="droptarget">
                    <div class="droparea">Nr of items: {{ items2.length }}</div>
                  </draggable>
                </v-list>
              </v-flex>
            </v-layout>
          </v-container>
        </v-content>
        </draggable>
        ''').tag(sync=True)

    css = Unicode('''
        /* Hide dragged element in target */
        .droptarget > [draggable=true] {
            display: none;
        }

        .sortable-ghost.no-drop {
           cursor: no-drop;
        }

        .droparea {
            height: 200px;
            border: 1px solid black;
        }
    ''').tag(sync=True)


    methods = Unicode('''{
        checkMove(e) {
            duplicate = this.items2.some(item => item.id === e.draggedContext.element.id)
            canMove = e.to.id !== "source" && !duplicate;
            cancelDrop = e.to.id === "catchAll"

            if (canMove && !cancelDrop) {
                e.dragged.classList.remove('no-drop')
            } else {
                e.dragged.classList.add('no-drop')
            }

            return canMove;
        },
    }''').tag(sync=True)
    

MyDraggableArea()